<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Verification_by_abstraction_with_eran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for Neural Network Verification with ERAN

---

*As an example we try to verifiy the **robustness** of a classification Network trained on the MNIST dataset.
We are using [Eran](https://github.com/eth-sri/eran).*

Important Links:
- 

---

# installation

In [1]:
%%capture
%%bash

git clone https://github.com/eth-sri/ERAN.git

sudo apt install m4
sudo apt install build-essential
sudo apt install autoconf
sudo apt install libtool
sudo apt install texlive-latex-base

In [2]:
%%bash

cd ERAN
sudo bash ./install.sh

gmp-6.1.2/
gmp-6.1.2/cxx/
gmp-6.1.2/mini-gmp/
gmp-6.1.2/Makefile.am
gmp-6.1.2/configure
gmp-6.1.2/acinclude.m4
gmp-6.1.2/configure.ac
gmp-6.1.2/aclocal.m4
gmp-6.1.2/gmpxx.h
gmp-6.1.2/Makefile.in
gmp-6.1.2/config.in
gmp-6.1.2/gmp-h.in
gmp-6.1.2/AUTHORS
gmp-6.1.2/COPYING
gmp-6.1.2/ChangeLog
gmp-6.1.2/INSTALL
gmp-6.1.2/NEWS
gmp-6.1.2/README
gmp-6.1.2/compile
gmp-6.1.2/config.guess
gmp-6.1.2/config.sub
gmp-6.1.2/install-sh
gmp-6.1.2/ltmain.sh
gmp-6.1.2/missing
gmp-6.1.2/gmp-impl.h
gmp-6.1.2/longlong.h
gmp-6.1.2/errno.c
gmp-6.1.2/assert.c
gmp-6.1.2/compat.c
gmp-6.1.2/extract-dbl.c
gmp-6.1.2/invalid.c
gmp-6.1.2/memory.c
gmp-6.1.2/mp_bpl.c
gmp-6.1.2/mp_clz_tab.c
gmp-6.1.2/mp_dv_tab.c
gmp-6.1.2/mp_minv_tab.c
gmp-6.1.2/mp_get_fns.c
gmp-6.1.2/mp_set_fns.c
gmp-6.1.2/version.c
gmp-6.1.2/nextprime.c
gmp-6.1.2/primesieve.c
gmp-6.1.2/tal-debug.c
gmp-6.1.2/tal-notreent.c
gmp-6.1.2/tal-reent.c
gmp-6.1.2/configfsf.guess
gmp-6.1.2/configfsf.sub
gmp-6.1.2/.gdbinit
gmp-6.1.2/INSTALL.autoconf
gmp-6.1.2/COPY

--2022-07-06 10:51:52--  https://gmplib.org/download/gmp/gmp-6.1.2.tar.xz
Resolving gmplib.org (gmplib.org)... 130.242.124.102
Connecting to gmplib.org (gmplib.org)|130.242.124.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1946336 (1.9M) [application/octet-stream]
Saving to: ‘gmp-6.1.2.tar.xz’

     0K .......... .......... .......... .......... ..........  2%  225K 8s
    50K .......... .......... .......... .......... ..........  5%  449K 6s
   100K .......... .......... .......... .......... ..........  7%  448K 5s
   150K .......... .......... .......... .......... .......... 10%  449K 5s
   200K .......... .......... .......... .......... .......... 13%  451K 4s
   250K .......... .......... .......... .......... .......... 15% 93.7M 4s
   300K .......... .......... .......... .......... .......... 18%  451K 3s
   350K .......... .......... .......... .......... .......... 21%  446K 3s
   400K .......... .......... .......... .......... .......... 2

In [3]:
%%bash

cd ERAN
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pycddlib: filename=pycddlib-2.1.6-cp37-cp37m-linux_x86_64.whl size=680802 sha256=ecebda86eedf6945be972754503d880ba3c272f77b7ab28cbbe19b6ccbd41eb8
  Stored in directory: /root/.cache/pip/wheels/c6/3e/5e/c7ac78b30857b829b06c0d00d3cac2429f97695193a86128b8
Successfully built pycddlib
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
%pip uninstall -y pillow
%pip install pillow

Found existing installation: Pillow 9.2.0
Uninstalling Pillow-9.2.0:
  Successfully uninstalled Pillow-9.2.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# imports

In [1]:
%%capture

%pip install onnx onnxruntime

import torch
from torch import nn
import numpy as np
import torch.onnx
import sys

import matplotlib.pyplot as plt

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/mnist_trainer.py'}

import mnist_trainer

# mnist

## define and train nn

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
%%capture
train_dataloader, test_dataloader = mnist_trainer.load_mnist_dataset(batch_size=64)

In [ ]:
model = mnist_trainer.train_model(
    NeuralNetwork(input_dim=28*28, output_dim=10, number_of_neurons=20), 
    epochs=3, 
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    )

Epoch 1
-------------------------------


100%|██████████| 938/938 [00:07<00:00, 118.58it/s]


Test Error: 
 Accuracy: 90.3%, Avg loss: 0.325855 

Epoch 2
-------------------------------


100%|██████████| 938/938 [00:07<00:00, 118.69it/s]


Test Error: 
 Accuracy: 91.8%, Avg loss: 0.273854 

Epoch 3
-------------------------------


100%|██████████| 938/938 [00:07<00:00, 120.76it/s]


Test Error: 
 Accuracy: 92.7%, Avg loss: 0.242850 

Done!


export model

In [ ]:
model_filename = "mnist_net.onnx"
dummy_input=torch.randn(1, 28, 28)

# set model to eval mode
model.eval()

# create a dummy input in the shape of the input values
device = "cuda" if torch.cuda.is_available() else "cpu"
dummy_input = dummy_input.to(device)

torch.onnx.export(model,
                  dummy_input,
                  model_filename,
                  export_params=True,
                  verbose=False,
                  input_names=['image'],
                  output_names=['classification'],
                  )

## abstract verification

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --epsilon 0.1 --domain deepzono --dataset mnist

{'approx_k': True,
 'attack': False,
 'complete': False,
 'data_dir': None,
 'dataset': 'mnist',
 'debug': False,
 'delta': 0.3,
 'domain': 'deepzono',
 'epsfile': None,
 'epsilon': 0.1,
 'from_test': 0,
 'gamma': inf,
 'geometric': False,
 'geometric_config': None,
 'input_box': None,
 'k': 3,
 'logdir': None,
 'logname': None,
 'max_milp_neurons': 30,
 'mean': None,
 'n_milp_refine': 1,
 'netname': '../../mnist_net.onnx',
 'normalized_region': True,
 'num_params': 0,
 'num_tests': None,
 'numproc': 2,
 'output_constraints': None,
 'partial_milp': 0,
 'quant_step': None,
 'refine_neurons': False,
 's': -2,
 'sparse_n': 70,
 'spatial': False,
 'specnumber': None,
 'std': None,
 'subset': None,
 't_norm': 'inf',
 'target': None,
 'timeout_complete': None,
 'timeout_final_lp': 100,
 'timeout_final_milp': 100,
 'timeout_lp': 1,
 'timeout_milp': 1,
 'use_default_heuristic': True,
 'use_milp': True,
 'vnn_lib_spec': None,
 'zonotope': None}
netname  ../../mnist_net.onnx  epsilon  0.1  domai

2022-07-04 09:50:24.529822: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
./optimizer.py:26: UserWarning: gpupoly not available.
  warnings.warn("gpupoly not available.")


In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../ffnnRELU__PGDK_w_0_3_6_500.onnx --epsilon 0.1 --domain deepzono --dataset mnist

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --epsilon 0.03 --domain deepzono --dataset mnist

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --epsilon 0.03 --domain deepzono --from_test 240 --dataset mnist

In [ ]:
%%bash
wget --no-cache --backups=1 {'https://github.com/eth-sri/eran/files/3653882/zonotope_example.txt'}

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --zonotope ../../zonotope_example.txt --domain deepzono --dataset mnist

In [ ]:
%%bash

cd ERAN/tf_verify/
python3 . --netname ../../mnist_net.onnx --dataset mnist --domain deeppoly --spatial --t-norm inf --delta 3

In [ ]:
%%bash

#wget --no-cache --backups=1 {'https://files.sri.inf.ethz.ch/eran/nets/onnx/mnist/mnist_relu_3_50.onnx'}

cd ERAN/tf_verify/

python3 . --netname ../../mnist_relu_3_50.onnx --epsilon 0.1 --domain deepzono --dataset mnist

In [ ]:
!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/eth-sri/deepg/master/code/examples/example1/config.txt'}

In [ ]:
%%bash

cd ERAN

set -e

cd gurobi912/linux64/src/build
sed -ie 's/^C++FLAGS =.*$/& -fPIC/' Makefile
make
cp libgurobi_c++.a ../../lib/
cp ../../lib/libgurobi90.so /usr/lib
cd ..

export GUROBI_HOME="$(pwd)/gurobi900/linux64"
export PATH="${PATH}:${GUROBI_HOME}/bin"
export CPATH="${CPATH}:${GUROBI_HOME}/include"
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib:${GUROBI_HOME}/lib

git clone https://github.com/eth-sri/deepg.git
cd deepg/code
mkdir build
make shared_object
cp ./build/libgeometric.so /usr/lib
cd ../..

export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib:/usr/lib

ldconfig

Footer

cd tf_verify/

python3 . --netname ../../mnist_net.onnx --geometric --geometric_config ../../config.txt --num_params 1 --dataset mnist

# iris

In [2]:
%%capture

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/iris_trainer.py'}
import iris_trainer

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/plots.py'}
import plots

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
names, feature_names, X, y, X_scaled, X_train, X_test, y_train, y_test = iris_trainer.load_dataset()

model = iris_trainer.train_model(
    NeuralNetwork(input_dim=X_train.shape[1], output_dim=3, number_of_neurons=5), 
    epochs=100, 
    X_train=X_train, 
    X_test=X_test, 
    y_train=y_train, 
    y_test=y_test)

Shape of X (data): (150, 4)
Shape of y (target): (150,) int64
Example of x and y pair: [5.1 3.5 1.4 0.2] 0
Shape of training set X (120, 4)
Shape of test set X (30, 4)


100%|██████████| 100/100 [00:00<00:00, 551.46it/s]


Done. Accuracy: 0.8333333333333334


In [ ]:
plots.show_plots(names, feature_names, X, y, title = 'Ground Truth')
plots.show_plots(names, feature_names, X, iris_trainer.predict(X_scaled, model), title = 'Classification from our network')

In [5]:
model_filename = "iris_net.onnx"
dummy_input=torch.randn(1, 4)

# set model to eval mode
model.eval()

# create a dummy input in the shape of the input values
device = "cuda" if torch.cuda.is_available() else "cpu"
dummy_input = dummy_input.to(device)

torch.onnx.export(model,
                  dummy_input,
                  model_filename,
                  export_params=True,
                  verbose=False,
                  input_names=['data'],
                  output_names=['classification'],
                  )

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --epsilon 0.1 --domain deepzono --dataset mnist